# Script for creating database and DW, its tables and pushing data to these tables

### 1. Running sub-scripts

In [2]:
%run Weather.ipynb
%run Crime_final.ipynb

### Configurations

In [16]:
import pyodbc
import pandas as pd
from warnings import simplefilter

WEATHER_FILE = 'Weather_final.csv'
CRIME_FILE = 'CRIME_FILE.csv'

SERVER = 'localhost'
DATABASE = 'Miniprojekti2'
DATAWAREHOUSE = 'Miniprojekti2DW'
USER = 'sa'
PWD = 'Password1'

simplefilter(action='ignore', category=UserWarning)

### 2. Create DB if not exist

In [17]:
try:
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';UID='+USER+';PWD='+ PWD, autocommit=True)
    cursor = cnxn.cursor()
    cursor.execute("CREATE DATABASE Miniprojekti2")
    cnxn.commit()
    cursor.close()
    cnxn.close()
except:
    print('Database already exists!')

Database already exists!


### 3. Create Weather table if not exist

In [18]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
try:
    cursor.execute("""
    CREATE TABLE Weather(
        DateKey int Primary Key,
        Temperature float,
        Humidity float,
        Rainfall float,
        Clouds float,
        Wind float,
        Weather_time datetime)
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE Weather")

cnxn.commit()
cursor.close()
cnxn.close()

Table already exists!


### 4. Load data into Weather table

In [19]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
df_weather = pd.read_csv(WEATHER_FILE)
for index, row in df_weather.iterrows():
    cursor.execute("INSERT INTO dbo.Weather values(?,?,?,?,?,?,?)",
                   row['DateKey'],
                   row['Temperature'],
                   row['Humidity'],
                   row['Rainfall'],
                   row['Clouds'],
                   row['Wind'],
                   row['Weather_time']
                   )
    cnxn.commit()
cursor.close()
cnxn.close()

### 5. Create Crime table if not exist

In [20]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
try:
    cursor.execute("""
    CREATE TABLE Crimes(
        CrimeID int Primary Key,
        IncidentCategory varchar(250),
        Neighborhood varchar(250),
        Datetime datetime,
        Hour int,
        Weather int)
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE Crimes")

cnxn.commit()
cursor.close()
cnxn.close()

Table already exists!


### 6. Load data into Crime table

In [21]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
df = pd.read_csv(CRIME_FILE, sep=';')
for index, row in df.iterrows():
    cursor.execute('INSERT INTO dbo.Crimes values(?,?,?,?,?,?)',
                   index,
                   row['incidentCategory'],
                   row['neighborhood'],
                   row['datetime'],
                   row['hour'],
                   row['weather']
                   )
    cnxn.commit()
cursor.close()
cnxn.close()

### 7. Create DW if not exist

In [22]:
try:
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';UID='+USER+';PWD='+ PWD, autocommit=True)
    cursor = cnxn.cursor()
    cursor.execute("CREATE DATABASE Miniprojekti2DW")
    cnxn.commit()
    cursor.close()
    cnxn.close()
except:
    print('Database already exists!')

Database already exists!


### 8. Create FactCrimes table if not exist

In [23]:
cnxn_dest = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn_dest.cursor()
try:
    cursor.execute("""CREATE TABLE FactCrimes(
                        CrimeID int Primary Key,
                        IncidentCategory varchar(250),
                        Neighborhood varchar(250),
                        DateKey int,
                        Temperature float,
                        Humidity float,
                        Rainfall float,
                        Clouds float,
                        Wind float)
    """)
except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE FactCrimes")
    
cnxn_dest.commit()
cursor.close()
cnxn_dest.close()

Table already exists!


### 9. Transform data for DW

In [24]:
cnxn_src = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USER+';PWD='+ PWD)

df_dw = pd.read_sql("""
SELECT * 
FROM Crimes AS C
FULL JOIN Weather AS W ON C.Weather = W.DateKey
WHERE Weather BETWEEN 2022092501 AND 2023030712
""", con=cnxn_src)

# Set CrimeID as index
df_dw.set_index(['CrimeID'], inplace=True)
df_dw.drop(columns=['Weather','Weather_time'], inplace=True)
df_dw.to_csv('FactCrimes.csv')
df_dw.head()

IncidentCategory                    Neighborhood  \
CrimeID                                                        
16843          Larceny Theft                         Mission   
16844    Motor Vehicle Theft                    Hayes Valley   
16845                  Fraud                         Mission   
16846          Larceny Theft                       Chinatown   
16847     Malicious Mischief  Financial District/South Beach   
...                      ...                             ...   
46144    Motor Vehicle Theft           Bayview Hunters Point   
46145          Larceny Theft                  Bernal Heights   
46146                  Fraud                      Tenderloin   
46147          Larceny Theft  Financial District/South Beach   
46148    Motor Vehicle Theft           Bayview Hunters Point   

                   Datetime  Hour     DateKey  Temperature  Humidity  \
CrimeID                                                                
16843   2022-09-25 01:00:00     1  2022092501         15.5      97.0   
16844   2022-09-25 01:00:00     1  2022092501         15.5      97.0   
16845   2022-09-25 01:00:00     1  2022092501         15.5      97.0   
16846   2022-09-25 01:00:00     1  2022092501         15.5      97.0   
16847   2022-09-25 01:19:00     1  2022092501         15.5      97.0   
...                     ...   ...         ...          ...       ...   
46144   2023-03-07 11:55:00    11  2023030711          9.0      68.0   
46145   2023-03-07 12:00:00    12  2023030712         10.2      62.0   
46146   2023-03-07 12:00:00    12  2023030712         10.2      62.0   
46147   2023-03-07 12:00:00    12  2023030712         10.2      62.0   
46148   2023-03-07 12:02:00    12  2023030712         10.2      62.0   

         Rainfall  Clouds  Wind  
CrimeID                          
16843         0.0     0.0  2.20  
16844         0.0     0.0  2.20  
16845         0.0     0.0  2.20  
16846         0.0     0.0  2.20  
16847         0.0     0.0  2.20  
...           ...     ...   ...  
46144         0.0    48.0  1.94  
46145         0.0    80.0  2.58  
46146         0.0    80.0  2.58  
46147         0.0    80.0  2.58  
46148         0.0    80.0  2.58  

[29306 rows x 10 columns]

### 10. Load FactCrimes

In [25]:
cnxn_dest = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn_dest.cursor()

for index,row in df_dw.iterrows():
    cursor.execute('INSERT INTO dbo.FactCrimes values (?,?,?,?,?,?,?,?,?)',
                   index,
                   row['IncidentCategory'],
                   row['Neighborhood'],
                   row['DateKey'],
                   row['Temperature'],
                   row['Humidity'],
                   row['Rainfall'],
                   row['Clouds'],
                   row['Wind'])
    cnxn_dest.commit()
cursor.close()
cnxn_dest.close()

### 11. Create DimDate table if not exist

In [26]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()
try:
    cursor.execute("""
    CREATE TABLE DimDate(
        DateKey int Primary Key,
        Datetime datetime,
        Year int,
        Month int,
        Day int,
        Hour int)
    """)

except:
    print('Table already exists!')
    cursor.execute("TRUNCATE TABLE DimDate")
    
cnxn.commit()
cursor.close()
cnxn.close()

Table already exists!


### 12. Generate DimDate

In [27]:
%run DimDate.ipynb

### 13. Load DimDate into DW

In [28]:
df_dd = pd.read_csv('DimDate.csv')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATAWAREHOUSE+';UID='+USER+';PWD='+ PWD)
cursor = cnxn.cursor()

for index, row in df_dd.iterrows():
    cursor.execute("INSERT INTO DimDate values(?,?,?,?,?,?)", 
                   int(row['DateKey']), 
                   row['Datetime'], 
                   int(row['Year']), 
                   int(row['Month']), 
                   int(row['Day']), 
                   int(row['Hour']))
    cnxn.commit()
    
cursor.close()
cnxn.close()